In [ ]:
import pandas as pd
import numpy  as np
from math import sin, cos, pi, sqrt
import random
import time
import sys
import rtsvg
rt = rtsvg.RACETrack()

# mostly a duplicate of the circle_routing...

_n_paths_         = 50
_n_circles_       = 100
_radius_min_      = 20
_radius_max_      = 30
_circle_geoms_    = []
_min_circle_sep_  = 40
_half_sep_        = _min_circle_sep_/2.0   # Needs to be more than the _radius_inc_test_
_radius_inc_test_ = 4
_radius_start_    = _radius_inc_test_ + 1  # Needs to be more than the _radius_inc_test_ ... less than the _min_circle_sep_
_escape_px_       = 20                     # less than the _min_circle_sep_

def circleOverlaps(cx, cy, r):
    for _geom_ in _circle_geoms_:
        dx, dy = _geom_[0] - cx, _geom_[1] - cy
        d      = sqrt(dx*dx+dy*dy)
        if d < (r + _geom_[2] + _min_circle_sep_): # at least 10 pixels apart...
            return True
    return False

def findOpening():
    _max_attempts_ = 100
    attempts  = 0
    cx, cy, r = random.randint(_radius_max_+_min_circle_sep_, 600-_radius_max_-_min_circle_sep_),     random.randint(_radius_max_+_min_circle_sep_, 400-_radius_max_-_min_circle_sep_), random.randint(_radius_min_,_radius_max_)
    while circleOverlaps(cx,cy,r) and attempts < _max_attempts_:
        cx, cy, r = random.randint(_radius_max_+_min_circle_sep_, 600-_radius_max_-_min_circle_sep_), random.randint(_radius_max_+_min_circle_sep_, 400-_radius_max_-_min_circle_sep_), random.randint(_radius_min_,_radius_max_)
        attempts += 1
    if attempts == _max_attempts_:
        return None
    return cx, cy, r

for i in range(_n_circles_):
    to_unpack = findOpening()
    if to_unpack is not None:
        cx, cy, r = to_unpack
        _circle_geoms_.append((cx,cy,r))

# points to connect
_pts_ = []
c0    = random.randint(0, len(_circle_geoms_)-1)
c1    =  random.randint(0, len(_circle_geoms_)-1)
while c1 == c0:
    c1 =  random.randint(0, len(_circle_geoms_)-1)
a0, a1 = random.random() * 2 * pi, random.random() * 2 * pi
cx, cy, r = _circle_geoms_[c0]
_pts_.append((cx+(r+_radius_start_+1)*cos(a0), cy+(r+_radius_start_+1)*sin(a0)))
_pts_.append((cx+(r+_escape_px_)*cos(a0), cy+(r+_escape_px_)*sin(a0)))
cx, cy, r = _circle_geoms_[c1]
_pts_.append((cx+(r+_escape_px_)*cos(a1), cy+(r+_escape_px_)*sin(a1)))
_pts_.append((cx+(r+_radius_start_+1)*cos(a1), cy+(r+_radius_start_+1)*sin(a1)))

entries         = [_pts_]
_exit_segments_ = []
_exit_segments_.append(_pts_[2:])
for i in range(_n_paths_):
    _more_pts_ = [_pts_[0], _pts_[1]]
    c          = random.randint(0,len(_circle_geoms_)-1)
    while c == c0:
        c  = random.randint(0,len(_circle_geoms_)-1)
    a = random.random() * 2 * pi
    cx, cy, r = _circle_geoms_[c]
    _more_pts_.append((cx+(r+_escape_px_)     *cos(a), cy+(r+_escape_px_)     *sin(a)))
    _more_pts_.append((cx+(r+_radius_start_+1)*cos(a), cy+(r+_radius_start_+1)*sin(a)))
    _exit_segments_.append(_more_pts_[2:])
    entries.append(_more_pts_)

In [ ]:
def calculatePathAroundCircles(pts, circle_geoms, radius_inc_test, half_sep):
    def breakSegment(_segment_):
        if rt.segmentLength(_segment_) < 2.0:
            return _segment_
        for _geom_ in circle_geoms:
            _circle_plus_ = (_geom_[0], _geom_[1], _geom_[2]+radius_inc_test)
            _dist_, _inter_  = rt.segmentIntersectsCircle(_segment_,_circle_plus_)
            if _dist_ <= _circle_plus_[2]:
                if _inter_[0] == _geom_[0] and _inter_[1] == _geom_[1]:
                    dx, dy   = _segment_[1][0] - _segment_[0][0], _segment_[1][1] - _segment_[0][1]
                    l        = sqrt(dx*dx+dy*dy)
                    dx,  dy  = dx/l, dy/l
                    pdx, pdy = -dy, dx 
                    return [(_segment_[0][0], _segment_[0][1]), (_geom_[0] + pdx*(_geom_[2]+half_sep), _geom_[1] + pdy*(_geom_[2]+half_sep)), (_segment_[1][0], _segment_[1][1])]
                else:
                    dx, dy = _inter_[0] - _geom_[0], _inter_[1] - _geom_[1]
                    l      = sqrt(dx*dx+dy*dy)
                    dx, dy = dx/l, dy/l
                    return [(_segment_[0][0], _segment_[0][1]), (_geom_[0]  + dx*(_geom_[2]+half_sep), _geom_[1]  + dy*(_geom_[2]+half_sep)), (_segment_[1][0], _segment_[1][1])]
        return _segment_

    last_length = 0
    _segments_  = []
    for _pt_ in pts:
        _segments_.append(_pt_)
    while last_length != len(_segments_):
        last_length    = len(_segments_)
        _new_segments_ = []
        for i in range(len(_segments_)-1):
            _new_ = breakSegment([_segments_[i], _segments_[i+1]])
            if len(_new_) == 3:
                _new_segments_.append(_new_[0])
                _new_segments_.append(_new_[1])
            else:
                _new_segments_.append(_new_[0])
        _new_segments_.append(_new_[-1])
        _segments_ = _new_segments_
        
    return _segments_

all_paths = []
for x in entries:
    _segments_ = calculatePathAroundCircles(x, _circle_geoms_, _radius_inc_test_, _half_sep_)
    all_paths.append(_segments_)

svg = '<svg x="0" y="0" width="600" height="400"><rect x="0" y="0" width="600" height="400" fill="#ffffff" />'
# Render Cirlces
for _geom_ in _circle_geoms_:
    svg += f'<circle cx="{_geom_[0]}" cy="{_geom_[1]}" r="{_geom_[2]}" stroke="#000000" fill="#000000" fill-opacity="0.2" />'
# Render Paths
for _path_ in all_paths:
    for i in range(len(_path_)-1):
        svg += f'<line x1="{_path_[i][0]}" y1="{_path_[i][1]}" x2="{_path_[i+1][0]}" y2="{_path_[i+1][1]}" stroke="#000000" stroke-width="0.2" />'
# Render Entry Points
svg += f'<line   x1="{entries[0][0][0]}" y1="{entries[0][0][1]}" x2="{entries[0][1][0]}" y2="{entries[0][1][1]}" stroke="#00af00" stroke-width="2" />'
svg += f'<circle cx="{entries[0][0][0]}" cy="{entries[0][0][1]}" r="3" stroke="#00af00" fill="#00af00" />'
for _entry_ in entries:
    svg += f'<line   x1="{_entry_[2][0]}" y1="{_entry_[2][1]}" x2="{_entry_[3][0]}" y2="{_entry_[3][1]}" stroke="#ff0000" stroke-width="2" />'
    svg += f'<circle cx="{_entry_[3][0]}" cy="{_entry_[3][1]}" r="3" stroke="#ff0000" fill="#ff0000" />'
svg += '</svg>'

svg2 = '<svg x="0" y="0" width="600" height="400"><rect x="0" y="0" width="600" height="400" fill="#ffffff" />'
# Render Cirlces
for _geom_ in _circle_geoms_:
    svg2 += f'<circle cx="{_geom_[0]}" cy="{_geom_[1]}" r="{_geom_[2]}" stroke="#000000" fill="#000000" fill-opacity="0.2" />'
# Render Paths
for _path_ in all_paths:
    _path_ = rt.expandSegmentsIntoPiecewiseCurvedParts(_path_, ampends=10.0, amp=10.0)
    for i in range(len(_path_)-1):
        svg2 += f'<line x1="{_path_[i][0]}" y1="{_path_[i][1]}" x2="{_path_[i+1][0]}" y2="{_path_[i+1][1]}" stroke="#000000" stroke-width="0.2" />'
# Render Entry Points
svg2 += f'<line   x1="{entries[0][0][0]}" y1="{entries[0][0][1]}" x2="{entries[0][1][0]}" y2="{entries[0][1][1]}" stroke="#00af00" stroke-width="2" />'
svg2 += f'<circle cx="{entries[0][0][0]}" cy="{entries[0][0][1]}" r="3" stroke="#00af00" fill="#00af00" />'
for _entry_ in entries:
    # svg2 += f'<line   x1="{_entry_[2][0]}" y1="{_entry_[2][1]}" x2="{_entry_[3][0]}" y2="{_entry_[3][1]}" stroke="#ff0000" stroke-width="2" />'
    svg2 += f'<circle cx="{_entry_[3][0]}" cy="{_entry_[3][1]}" r="3" stroke="#ff0000" fill="#ff0000" />'
svg2 += '</svg>'

rt.tile([svg,svg2])

In [ ]:
qt      = rt.xyQuadTree((0.0, 0.0, 1.0, 1.0), max_pts_per_node=50)
pts     = []
t_setup = 0
for s in range(10000):
    pt = (random.random(),random.random())
    pts.append(pt)
    t0 = time.time()
    qt.add([pt])
    t1 = time.time()
    t_setup += (t1-t0)
print(f'QT Setup = {t_setup:0.2f}s')

overall_time_raw, overall_time_qt, calcs = 0.0, 0.0, 0
for n in range(1,10):
    _errors_ = 0
    time_raw, time_qt, _samples_ = 0.0, 0.0, 5000
    for _test_ in range(_samples_):
        p           = (random.random(), random.random())
        sorter_full = []
        t0 = time.time()
        for pt in pts:
            sorter_full.append((rt.segmentLength((p, pt)), pt))
        sorter_full  = sorted(sorter_full)[:n]
        t1 = time.time()
        sorter_qtree = qt.closest(p,n=n)
        t2 = time.time()
        in_err = False
        for i in range(len(sorter_full)):
            if sorter_full[i] != sorter_qtree[i]:
                in_err = True
        if in_err:
            _errors_ += 1
        time_raw += (t1-t0)
        time_qt  += (t2-t1)
        calcs    += 1
    print(f'{n} : Errors Found = {_errors_}/{_samples_} | Raw = {(time_raw):0.2f}s | QT = {(time_qt):0.2f}s')
    overall_time_raw += time_raw
    overall_time_qt  += time_qt

print(f'Total    | Raw = {(overall_time_raw):0.2f}s | QT = {(overall_time_qt):0.2f}s')
print(f'Per Calc | Raw = {(overall_time_raw/calcs):0.4f}s | QT = {(overall_time_qt/calcs):0.4f}s')

In [ ]:
# 2023-02-23 19:34 (5000 samples per n) ... setup was max_pts = 50
# ... the data is random... so probably doensn't match realworld use...
# ... used the __nbors__STRICT_
example_results_just_exact_no_diagonal_neighbors = '''
QT Setup = 0.01s
1 : Errors Found = 0/5000  | Raw = 13.08s | QT = 0.51s
2 : Errors Found = 0/5000  | Raw = 13.10s | QT = 0.51s
3 : Errors Found = 1/5000  | Raw = 13.18s | QT = 0.50s
4 : Errors Found = 5/5000  | Raw = 13.20s | QT = 0.50s
5 : Errors Found = 2/5000  | Raw = 13.20s | QT = 0.51s
6 : Errors Found = 9/5000  | Raw = 13.22s | QT = 0.50s
7 : Errors Found = 10/5000 | Raw = 13.28s | QT = 0.51s
8 : Errors Found = 15/5000 | Raw = 13.37s | QT = 0.51s
9 : Errors Found = 18/5000 | Raw = 13.38s | QT = 0.51s
Total    | Raw = 119.01s | QT = 4.56s
Per Calc | Raw = 0.0026s | QT = 0.0001s
'''
# 2023-02-24 09:44 (5000 samples per n) ... setup was max_pts = 50
# ... same issue with random data -- may not mirror real world results
# ... used the new __nbors__ which uses an error bar of 10% to get diagonals...
# ... and includes a check to not split nodes that can't be split
# ... (can't be split because maybe the supplied tuple has the same exact x and y but different 3rd... 4th... elements)
example_results_just_exact_some_diagonal_neighbors = '''
QT Setup = 0.01s
1 : Errors Found = 0/5000 | Raw = 13.08s | QT = 0.53s
2 : Errors Found = 0/5000 | Raw = 13.10s | QT = 0.53s
3 : Errors Found = 0/5000 | Raw = 13.04s | QT = 0.53s
4 : Errors Found = 0/5000 | Raw = 13.09s | QT = 0.53s
5 : Errors Found = 1/5000 | Raw = 13.22s | QT = 0.54s
6 : Errors Found = 1/5000 | Raw = 13.22s | QT = 0.54s
7 : Errors Found = 1/5000 | Raw = 13.25s | QT = 0.54s
8 : Errors Found = 1/5000 | Raw = 13.23s | QT = 0.53s
9 : Errors Found = 7/5000 | Raw = 13.23s | QT = 0.54s
Total    | Raw = 118.46s | QT = 4.80s
Per Calc | Raw = 0.0026s | QT = 0.0001s
'''